In [1]:
seed_value= 0

import os
os.environ['PYTHONHASHSEED']=str(seed_value)

import random
random.seed(seed_value)

import numpy as np
np.random.seed(seed_value)

import tensorflow as tf

from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passi

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt
import pickle

#Scikit-Learn Importing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import make_scorer
from sklearn.svm import SVR

#Import PyWavelets for WaveSmoothing
import pywt
import mad
from statsmodels import robust

#Import Keras
import keras
from keras.layers import Input
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.optimizers import Adadelta
from tqdm import tqdm
from keras import backend as K

#Import Gridsearch
from hypopt import GridSearch

#Import Capstone Functions
from Capstone_Functions import *
cf = CapstoneFunctions()

In [3]:
Monthly = pd.read_pickle('data/Monthly.pkl')
Monthly2 = pd.read_pickle('data/MonthlyReturns.pkl')

In [4]:
WOMonthly = Monthly.iloc[:,12:13].copy()
WOMonthly.tail()

,high3
date,
2020-03-02,-0.041491
2020-04-01,-0.123905
2020-05-01,0.078188
2020-06-01,0.048290
2020-07-01,-0.011371


In [5]:
FIPMonthly = Monthly2.iloc[:,16:17].copy()
FIPMonthly.head()

,FIPMonthly
date,
1990-12-03,0.034620
1991-01-01,0.022021
1991-02-01,0.013218
1991-03-01,0.089188
1991-04-01,0.031861


In [6]:
#Create Train_Test_Split for High and Low Momentum returns with a 60 Month Lookback
WOXdata, WOYdata, WOXtrain, WOYtrain, WOXtest, WOYtest, WOPredictedX, WOForecastX, WOX, WOY = \
cf.Train_Test_Split(WOMonthly, 0, 100, 12, scaler = False)
FIPXdata, FIPYdata, FIPXtrain, FIPYtrain, FIPXtest, FIPYtest, FIPPredictedX, FIPForecastX, FIPX, FIPY = \
cf.Train_Test_Split(FIPMonthly, 0, 100, 12, scaler = False)

In [7]:
def waveletSmooth(x, wavelet="haar", level=2, DecLvl=2):
    # calculate the wavelet coefficients
    coeff = pywt.wavedec(x, wavelet, mode="per", level=DecLvl)
    # calculate a threshold
    sigma = robust.mad(coeff[-level])
    uthresh = sigma * np.sqrt( 2*np.log( len( x )))
    coeff[1:] = (pywt.threshold( i, value=uthresh, mode="soft" ) for i in coeff[1:])
    # reconstruct the signal using the thresholded coefficients
    y = pywt.waverec(coeff, wavelet, mode="per")
    return y

In [8]:
wav_wo = waveletSmooth(WOMonthly.iloc[:,0])
wav_wo = wav_wo.reshape(len(wav_wo),1)

wav_fip = waveletSmooth(FIPMonthly.iloc[:,0])
wav_fip = wav_fip.reshape(len(wav_fip),1)

In [9]:
    def Train_Test_Split2(Data1, Hi_or_Lo, Window, Lookback, scaler): 

        """ 
        Import Monthly data with Date as Index. Select which portfolio to train test split. Set number of
        lookback months and whether to use MinMaxScaler.

        Example:
        HiXdata, HiYdata, HiXtrain, HiYtrain, HiXtest, HiYtest, HiPredictedX, HiForecastX, HiX, HiY = \
        Train_Test_Split(Monthly, 'Hi', 60, 12, scaler = False)

        """ 


        #If High or Low Momentum Portfolio
        if Hi_or_Lo == 'Hi':
            Port = 0
        else:
            Port =  0

        TrainSplit = int(Window*0.9)
        TestSplit = int(Window*0.1)

        #Iterate Through Data and Creat Numpy arrays with 12 Months of \ 
        #lagged data for X and 13th Month for Y
        tmpX=[]
        tmpY=[]
        for A in range(len(Data1)-Lookback):
            tmp=Data1[A:(A + Lookback),Port]
            tmpX.append(tmp)
            tmpY.append(Data1[(A + Lookback),Port])
        Xdata = np.array(tmpX)
        Ydata = np.array(tmpY)

        tmpForeX=[]
        for A in range(len(Data1)-Lookback):
            tmpFore=Data1[A+1:(A+1 + Lookback),Port]
            tmpForeX.append(tmpFore)
        XdataFore = np.array(tmpForeX)

        Ydata = Ydata.reshape(len(Ydata),1)

        Shape = int(Xdata.shape[0])
        Shape2 = int(XdataFore.shape[0])


        #Create Empty Numpy Arrays
        Xtrain=np.ones(shape=(Shape-Window,TrainSplit,12))
        Ytrain=np.ones(shape=(Shape-Window,TrainSplit,1))
        Xtest=np.ones(shape=(Shape-Window,TestSplit,Lookback))
        Ytest=np.ones(shape=(Shape-Window,TestSplit,1))
        PredictedX=np.ones(shape=(Shape-Window,1,Lookback))
        ForecastX=np.ones(shape=(Shape2-Window,1,Lookback))
        X = np.ones(shape=(Shape-Window,Window,Lookback))
        Y = np.ones(shape=(Shape-Window,Window,1))

        #Fill Numpy Arrays with data
        for B in range(Shape-Window):
            Xtrain[B,:,:] = Xdata[B:B+TrainSplit,:]
            Ytrain[B,:,:] = Ydata[B:B+TrainSplit]
            Xtest[B,:,:] = Xdata[B+TrainSplit:B+Window,:]
            Ytest[B,:,:] = Ydata[B+TrainSplit:B+Window]
            PredictedX[B,:,:] = Xdata[B+Window,:]
            ForecastX[B,:,:] = XdataFore[B+Window,:]
            X[B,:,:] = Xdata[B:B+Window,:]
            Y[B,:,:] = Ydata[B:B+Window]

            if scaler:
                sc = MinMaxScaler()
                Xtrain[B,:,:] = sc.fit_transform(Xtrain[B,:,:])
                Xtest[B,:,:] = sc.transform(Xtest[B,:,:])
                PredictedX[B,:,:] = sc.transform(PredictedX[B,:,:])
                X[B,:,:] = sc.transform(X[B,:,:])



        return Xdata, Ydata, Xtrain, Ytrain, Xtest, Ytest, PredictedX, ForecastX, X, Y

In [10]:
#Create Train_Test_Split for High and Low Momentum returns with a 100 Month Lookback
W_WO_Xdata, W_WO_Ydata, W_WO_Xtrain, W_WO_Ytrain, W_WO_Xtest, W_WO_Ytest, W_WO_ForecastX, \
W_WO_ForecastX, W_WO_X, W_WO_Y = \
Train_Test_Split2(wav_wo, 'Hi', 100, 12, scaler = False)

#Create Train_Test_Split for High and Low Momentum returns with a 100 Month Lookback
W_FIP_Xdata, W_FIP_Ydata, W_FIP_Xtrain, W_FIP_Ytrain, W_FIP_Xtest, W_FIP_Ytest, W_FIP_ForecastX,\
W_FIP_ForecastX, W_FIP_X, W_FIP_Y = \
Train_Test_Split2(wav_fip, 'Hi', 100, 12, scaler = False)

# DNN-using SgAE

In [11]:
epochs = 200
batch = 100

In [12]:
noise = [0.1,0.15,0.3,0.5]
def gaussian_noise(Xdata, noise_param):
    noise = np.random.normal(loc=0, scale=noise_param, size=Xdata.shape)
    Xdata = Xdata + noise
    return Xdata

In [13]:
def SdAE_Gauss(Xdata, Ydata, Xtrain, Ytrain, Xtest, Ytest, TruedataX, noise):
    keras.backend.clear_session()
    dataXd = gaussian_noise(Xdata, noise)
    input_1 = Input(shape=(12,))
    encoder_1 = Dense(6, activation = 'sigmoid') (input_1)
    
    x = Dropout(0.1) (encoder_1)
    
    batch_1 = BatchNormalization() (x)
    
    decoder_1 = Dense(12, activation = 'sigmoid') (batch_1)
    E1 = Model(input_1, encoder_1)
    D1 = Model(input_1, decoder_1)
    D1.compile(loss='mean_squared_error', optimizer=Adam(lr=0.5))
    D1.fit(Xdata, Xdata, batch_size=batch, \
           epochs=epochs, verbose = 0, validation_split = 0.1)

    retrieve_h1 = K.function([E1.layers[1].input], [E1.layers[1].output])
    
    hidden_1 = retrieve_h1([Xdata])[0]
    
    hidden_1 = gaussian_noise(hidden_1, noise)

    input_2 = Input(shape=(6,))
    encoder_2 = Dense(6, activation = 'sigmoid') (input_2)
    
    x = Dropout(0.1) (encoder_2)
    
    batch_2 = BatchNormalization() (x)
    
    decoder_2 = Dense(6, activation = 'sigmoid') (batch_2)
    E2 = Model(input_2, encoder_2)
    D2 = Model(input_2, decoder_2)
    D2.compile(loss='mean_squared_error', optimizer=Adam(lr=0.5))
    D2.fit(hidden_1, hidden_1, batch_size=batch, \
           epochs=epochs, verbose = 0, validation_split = 0.1)

    retrieve_h2 = K.function([E2.layers[1].input],[E2.layers[1].output])
    
    hidden_2 = retrieve_h2([hidden_1])[0]
        
    hidden_2 = gaussian_noise(hidden_2, noise)

    input_3 = Input(shape=(6,))
    encoder_3 = Dense(6, activation = 'sigmoid') (input_3)

    x = Dropout(0.1) (encoder_3)
    
    batch_3 = BatchNormalization() (encoder_3)
    decoder_3 = Dense(6, activation = 'sigmoid') (batch_3)
    E3 = Model(input_3, encoder_3)
    D3 = Model(input_3, decoder_3)
    D3.compile(loss='mean_squared_error', optimizer=Adam(lr=0.5))
    D3.fit(hidden_2, hidden_2, batch_size=batch, \
           epochs=epochs, verbose = 0, validation_split = 0.1)

    Fine_Tuneing = Sequential()
    Xtrain = gaussian_noise(Xtrain, noise)
    
    for X in E1.layers:
        Fine_Tuneing.add(X)
        Fine_Tuneing.add(Dropout(0.1))
        Fine_Tuneing.add(BatchNormalization())
    
    for X in E2.layers:
        Fine_Tuneing.add(X)
        Fine_Tuneing.add(Dropout(0.1))
        Fine_Tuneing.add(BatchNormalization())
    
    for X in E3.layers:
        Fine_Tuneing.add(X)
        Fine_Tuneing.add(Dropout(0.1)) 
        Fine_Tuneing.add(BatchNormalization())
    
    for X in Fine_Tuneing.layers:
        X.trainable = True
    
    Fine_Tuneing.add(Dense(units=1, activation='linear', input_shape=(6,)))
    Fine_Tuneing.compile(loss='mean_squared_error', optimizer=Adam(lr=0.5))
    Fine_Tuneing.fit(Xtrain, Ytrain, validation_data = (Xtest, Ytest), shuffle = False,\
    epochs=epochs, batch_size = batch, verbose = 0) 
    prediction = Fine_Tuneing.predict(TruedataX)
    Fine_Tuneing.summary()
    
    return (prediction, Fine_Tuneing)

In [14]:
def predictSdAEGauss(Xdata, Ydata, Xtrain, Ytrain, Xtest, Ytest, XFore):
    Final_Prediction = np.zeros(shape=(244,1))
    for i in range(Final_Prediction.shape[0]):
        mse = 1
        for y in noise:
            pred, model = SdAE_Gauss(Xdata[i], Ydata[i],\
            Xtrain[i], Ytrain[i], Xtest[i], Ytest[i], Xtest[i], y)
            if mean_squared_error(pred, Ytest[i]) < mse:
                best = model
                Final_Prediction[i] = best.predict(XFore[i])
            print('Noise: ',y)
        print('Prediction',i+1,'of',244)
    return Final_Prediction

In [15]:
Forecast_FIP_SdAE_Gauss = \
predictSdAEGauss(W_FIP_X, FIPY, W_FIP_Xtrain, FIPYtrain, W_FIP_Xtest, FIPYtest, W_FIP_ForecastX)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_4 (Dropout)          (None, 12)                0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 12)                48        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 78        
_________________________________________________________________
dropout_5 (Dropout)          (None, 6)                 0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 6)                 24        
_________________________________________________________________
input_2 (InputLayer)         (None, 6)                 0         


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_4 (Dropout)          (None, 12)                0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 12)                48        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 78        
_________________________________________________________________
dropout_5 (Dropout)          (None, 6)                 0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 6)                 24        
_________________________________________________________________
input_2 (InputLayer)         (None, 6)                 0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 6)                

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_4 (Dropout)          (None, 12)                0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 12)                48        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 78        
_________________________________________________________________
dropout_5 (Dropout)          (None, 6)                 0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 6)                 24        
_________________________________________________________________
input_2 (InputLayer)         (None, 6)                 0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 6)                

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_4 (Dropout)          (None, 12)                0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 12)                48        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 78        
_________________________________________________________________
dropout_5 (Dropout)          (None, 6)                 0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 6)                 24        
_________________________________________________________________
input_2 (InputLayer)         (None, 6)                 0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 6)                

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_4 (Dropout)          (None, 12)                0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 12)                48        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 78        
_________________________________________________________________
dropout_5 (Dropout)          (None, 6)                 0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 6)                 24        
_________________________________________________________________
input_2 (InputLayer)         (None, 6)                 0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 6)                

...

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_4 (Dropout)          (None, 12)                0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 12)                48        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 78        
_________________________________________________________________
dropout_5 (Dropout)          (None, 6)                 0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 6)                 24        
_________________________________________________________________
input_2 (InputLayer)         (None, 6)                 0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 6)                

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_4 (Dropout)          (None, 12)                0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 12)                48        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 78        
_________________________________________________________________
dropout_5 (Dropout)          (None, 6)                 0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 6)                 24        
_________________________________________________________________
input_2 (InputLayer)         (None, 6)                 0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 6)                

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_4 (Dropout)          (None, 12)                0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 12)                48        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 78        
_________________________________________________________________
dropout_5 (Dropout)          (None, 6)                 0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 6)                 24        
_________________________________________________________________
input_2 (InputLayer)         (None, 6)                 0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 6)                

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_4 (Dropout)          (None, 12)                0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 12)                48        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 78        
_________________________________________________________________
dropout_5 (Dropout)          (None, 6)                 0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 6)                 24        
_________________________________________________________________
input_2 (InputLayer)         (None, 6)                 0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 6)                

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_4 (Dropout)          (None, 12)                0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 12)                48        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 78        
_________________________________________________________________
dropout_5 (Dropout)          (None, 6)                 0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 6)                 24        
_________________________________________________________________
input_2 (InputLayer)         (None, 6)                 0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 6)                

In [16]:
SdAE_Gauss_FIP_Forecast = pd.DataFrame(Forecast_FIP_SdAE_Gauss)

In [17]:
SdAE_Gauss_FIP_Forecast.to_pickle('data/SdAE_Gauss_FIP_Forecast.pkl')

In [18]:
stop

NameError: name 'stop' is not defined